In [ ]:
from google.colab import files
files.upload()


Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"saunakbhattacharjee","key":"aa9043ee4f089abccc223d043637171c"}'}

In [ ]:
import os
import shutil

os.makedirs("/root/.kaggle", exist_ok=True)

shutil.move("kaggle.json", "/root/.kaggle/kaggle.json")

os.chmod("/root/.kaggle/kaggle.json", 0o600)

In [ ]:
!pip install -q kaggle


In [ ]:
!kaggle datasets download -d raddar/chest-xrays-indiana-university

Dataset URL: https://www.kaggle.com/datasets/raddar/chest-xrays-indiana-university
License(s): Attribution-NonCommercial-NoDerivatives 4.0 International (CC BY-NC-ND 4.0)
 99% 13.0G/13.2G [00:54<00:01, 137MB/s] 
100% 13.2G/13.2G [00:54<00:00, 258MB/s]


In [ ]:
!unzip -qq /content/chest-xrays-indiana-university.zip -d iu_xray/


In [ ]:
!pip install transformers accelerate peft pillow einops bitsandbytes num2words


  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 40.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.5/163.5 kB 15.7 MB/s eta 0:00:00
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13706 sha256=8f1c552f7e201256df1091a9a56f515ef62de706125f4a0908473e23d631e604
  Stored in directory: /root/.cache/pip/wheels/1a/bf/a1/4cee4f7678c68c5875ca89eaccf460593539805c3906722228
Successfully built docopt


In [ ]:
root = "/content/iu_xray"
for path in os.listdir(root):
    print(path)


indiana_projections.csv
images
indiana_reports.csv


In [ ]:
import pandas as pd
import re
import glob

In [ ]:
reports_path = "/content/iu_xray/indiana_reports.csv"
metadata_path = "/content/iu_xray/indiana_projections.csv"

reports_df = pd.read_csv(reports_path)
meta_df = pd.read_csv(metadata_path)

reports_df.head(), meta_df.head()

(   uid                                               MeSH  \
 0    1                                             normal   
 1    2  Cardiomegaly/borderline;Pulmonary Artery/enlarged   
 2    3                                             normal   
 3    4  Pulmonary Disease, Chronic Obstructive;Bullous...   
 4    5  Osteophyte/thoracic vertebrae/multiple/small;T...   
 
                                             Problems  \
 0                                             normal   
 1                      Cardiomegaly;Pulmonary Artery   
 2                                             normal   
 3  Pulmonary Disease, Chronic Obstructive;Bullous...   
 4                         Osteophyte;Thickening;Lung   
 
                                                image  \
 0                          Xray Chest PA and Lateral   
 1                Chest, 2 views, frontal and lateral   
 2                          Xray Chest PA and Lateral   
 3  PA and lateral views of the chest XXXX, XXXX a... 

In [ ]:
def clean_report(text):
    if pd.isna(text):
        return ""
    text = str(text)

    text = re.sub(r"\s+", " ", text).strip()

    text = re.sub(r"[^\w\s,.%-/]", "", text)

    text = text.lower()

    return text

In [ ]:
reports_df["findings_clean"] = reports_df["findings"].apply(clean_report)
reports_df["impression_clean"] = reports_df["impression"].apply(clean_report)


In [ ]:
#print(reports_df.columns)

In [ ]:
#reports_df.head()

In [ ]:
df = meta_df.merge(
    reports_df,
    on="uid",
    how="inner"
)

#print(df.head())
print(len(df), "total image-report pairs")

7466 total image-report pairs


In [ ]:
df["report_text"] = df["findings_clean"].fillna("") + " " + df["impression_clean"].fillna("")
df["report_text"] = df["report_text"].str.strip()


In [ ]:
df[["findings_clean", "impression_clean", "report_text"]].head()


,findings_clean,impression_clean,report_text
0,the cardiac silhouette and mediastinum size ar...,normal chest x-xxxx.,the cardiac silhouette and mediastinum size ar...
1,the cardiac silhouette and mediastinum size ar...,normal chest x-xxxx.,the cardiac silhouette and mediastinum size ar...
2,borderline cardiomegaly. midline sternotomy xx...,no acute pulmonary findings.,borderline cardiomegaly. midline sternotomy xx...
3,borderline cardiomegaly. midline sternotomy xx...,no acute pulmonary findings.,borderline cardiomegaly. midline sternotomy xx...
4,,"no displaced rib fractures, pneumothorax, or p...","no displaced rib fractures, pneumothorax, or p..."


In [ ]:
print(meta_df.columns)

Index(['uid', 'filename', 'projection'], dtype='object')


In [ ]:
print(df.columns)

Index(['uid', 'filename', 'projection', 'MeSH', 'Problems', 'image',
       'indication', 'comparison', 'findings', 'impression', 'findings_clean',
       'impression_clean', 'report_text'],
      dtype='object')


In [ ]:
image_root = "/content/iu_xray/images/images_normalized"

def find_image_by_uid(uid):
    uid = str(uid)
    pattern = os.path.join(image_root, uid + "_*")
    matches = glob.glob(pattern)
    if len(matches) > 0:
        return matches
    return None

In [ ]:
df["image_paths"] = df["uid"].apply(find_image_by_uid)
df = df.dropna(subset=["image_paths"])


In [ ]:
df["num_images"] = df["image_paths"].apply(lambda x: len(x))
df["num_images"].value_counts()


,count
num_images,
2,6416
3,542
1,446
4,57
5,5


In [ ]:

df["image_path"] = df["image_paths"].apply(lambda x: x[0])
df = df.drop(columns=["image_paths"])


In [ ]:
df[["uid", "image_path"]].head(10)


,uid,image_path
0,1,/content/iu_xray/images/images_normalized/1_IM...
1,1,/content/iu_xray/images/images_normalized/1_IM...
2,2,/content/iu_xray/images/images_normalized/2_IM...
3,2,/content/iu_xray/images/images_normalized/2_IM...
4,3,/content/iu_xray/images/images_normalized/3_IM...
5,3,/content/iu_xray/images/images_normalized/3_IM...
6,4,/content/iu_xray/images/images_normalized/4_IM...
7,4,/content/iu_xray/images/images_normalized/4_IM...
8,5,/content/iu_xray/images/images_normalized/5_IM...
9,5,/content/iu_xray/images/images_normalized/5_IM...


In [ ]:
import timm
import torch
import torch.nn as nn
import transformers
from transformers import AutoModelForVision2Seq, AutoTokenizer, AutoModelForSeq2SeqLM
from transformers import AutoModelForCausalLM, AutoProcessor, AutoModelForImageTextToText
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import torchvision.transforms as T
from peft import get_peft_model, LoraConfig, TaskType


In [ ]:
image_transform = T.Compose([
    T.Resize((224, 224)),
    T.ToTensor(),
    T.Normalize(mean=[0.5, 0.5, 0.5],
                std=[0.5, 0.5, 0.5])
])


In [ ]:
def ensure_rgb(img):
    if img.mode != "RGB":
        return img.convert("RGB")
    return img


In [ ]:
def clean_report(text):
    if pd.isna(text):
        return ""
    text = str(text)

    # Remove multiple spaces
    text = re.sub(r"\s+", " ", text).strip()

    # Remove XXXX patterns - anonymization markers
    text = re.sub(r"\bxxxx\b", "", text, flags=re.IGNORECASE)
    text = re.sub(r"\bx{2,}\b", "", text, flags=re.IGNORECASE)

    text = re.sub(r"[^a-zA-Z0-9\s,.%-/]", "", text)

    text = re.sub(r"\s+", " ", text).strip()

    text = text.lower()

    return text

In [ ]:
import torch
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from transformers import AutoProcessor, AutoModelForCausalLM


In [ ]:
# GIT is much lighter and works well for medical imaging
model_name = "microsoft/git-base"
processor = AutoProcessor.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16
).cuda()

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/503 [00:00<?, ?B/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


tokenizer_config.json:   0%|          | 0.00/453 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

`torch_dtype` is deprecated! Use `dtype` instead!


model.safetensors:   0%|          | 0.00/707M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/141 [00:00<?, ?B/s]

In [ ]:
class GIT_XRayDataset(Dataset):
    def __init__(self, df, processor, max_length=128):
        self.df = df.reset_index(drop=True)
        self.processor = processor
        self.max_length = max_length

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]

        try:
            # Load and verify image
            image = Image.open(row["image_path"]).convert("RGB")

            # Ensure image is valid
            if image.size[0] == 0 or image.size[1] == 0:
                print(f"Invalid image at index {idx}")
                return self.__getitem__((idx + 1) % len(self.df))

            text = row["report_text"]

            # Make sure text is not empty
            if not text or len(text.strip()) == 0:
                text = "no findings"  # Default text

            # Process with GIT processor
            encoding = self.processor(
                images=image,
                text=text,
                return_tensors="pt",
                padding="max_length",
                max_length=self.max_length,
                truncation=True
            )

            # Verify no NaN in processed data
            pixel_values = encoding["pixel_values"].squeeze(0)
            if torch.isnan(pixel_values).any():
                print(f"NaN in processed image at index {idx}")
                return self.__getitem__((idx + 1) % len(self.df))

            return {
                "pixel_values": pixel_values,
                "input_ids": encoding["input_ids"].squeeze(0),
                "attention_mask": encoding["attention_mask"].squeeze(0),
                "labels": encoding["input_ids"].squeeze(0),
            }

        except Exception as e:
            print(f"Error loading index {idx}: {e}")
            return self.__getitem__((idx + 1) % len(self.df))

In [ ]:
from sklearn.model_selection import train_test_split

train_df, val_df = train_test_split(df, test_size=0.2, random_state=42)
# Create datasets
train_dataset = GIT_XRayDataset(train_df, processor)
val_dataset = GIT_XRayDataset(val_df, processor)

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True, num_workers=2)
val_loader = DataLoader(val_dataset, batch_size=8, shuffle=False, num_workers=2)

In [ ]:
from transformers import get_linear_schedule_with_warmup
from tqdm import tqdm


In [ ]:
# Reinitialize model (in case it's corrupted)
model = AutoModelForCausalLM.from_pretrained("microsoft/git-base").cuda()

# IMPORTANT: Set model to training mode and check for frozen layers
model.train()

# Verify parameters are trainable
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"Trainable parameters: {trainable_params:,}")
if trainable_params == 0:
    print("⚠️ No trainable parameters! Unfreezing model...")
    for param in model.parameters():
        param.requires_grad = True

# Conservative optimizer with lower learning rate
optimizer = torch.optim.AdamW(
    model.parameters(),
    lr=1e-5,  # Much lower learning rate
    weight_decay=0.01,
    eps=1e-8  # Numerical stability
)

epochs = 10
num_training_steps = epochs * len(train_loader)
scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=num_training_steps // 10,
    num_training_steps=num_training_steps
)

# Training loop with NaN detection
for epoch in range(epochs):
    model.train()
    total_loss = 0
    valid_batches = 0

    progress = tqdm(train_loader, desc=f"Epoch {epoch+1}")

    for batch_idx, batch in enumerate(progress):
        try:
            pixel_values = batch["pixel_values"].cuda()
            input_ids = batch["input_ids"].cuda()
            attention_mask = batch["attention_mask"].cuda()
            labels = batch["labels"].cuda()

            # Check for NaN in inputs
            if torch.isnan(pixel_values).any():
                print(f"NaN in pixel_values at batch {batch_idx}, skipping")
                continue

            # Mask padding tokens in labels
            labels = labels.clone()
            labels[labels == processor.tokenizer.pad_token_id] = -100

            # Forward pass
            outputs = model(
                pixel_values=pixel_values,
                input_ids=input_ids,
                attention_mask=attention_mask,
                labels=labels
            )

            loss = outputs.loss

            # Check for NaN loss
            if torch.isnan(loss) or torch.isinf(loss):
                print(f"\n⚠️ NaN/Inf loss at batch {batch_idx}")
                print(f"Pixel values stats: min={pixel_values.min():.4f}, max={pixel_values.max():.4f}")
                print(f"Labels: {labels[0][:10]}")
                optimizer.zero_grad()
                continue

            # Check if loss is too large
            if loss.item() > 100:
                print(f"\n⚠️ Very large loss ({loss.item():.2f}) at batch {batch_idx}, skipping")
                optimizer.zero_grad()
                continue

            # Backward pass
            loss.backward()

            # Gradient clipping (important!)
            grad_norm = torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)

            # Check for NaN gradients
            if torch.isnan(grad_norm) or torch.isinf(grad_norm):
                print(f"\n⚠️ NaN/Inf gradients at batch {batch_idx}")
                optimizer.zero_grad()
                continue

            # Optimizer step
            optimizer.step()
            scheduler.step()
            optimizer.zero_grad()

            total_loss += loss.item()
            valid_batches += 1

            progress.set_postfix({
                "loss": f"{loss.item():.4f}",
                "avg_loss": f"{total_loss/valid_batches:.4f}",
                "lr": f"{scheduler.get_last_lr()[0]:.2e}",
                "grad_norm": f"{grad_norm:.2f}"
            })

        except RuntimeError as e:
            print(f"\n⚠️ Runtime error at batch {batch_idx}: {e}")
            optimizer.zero_grad()
            torch.cuda.empty_cache()
            continue

    if valid_batches == 0:
        print("⚠️ No valid batches processed in this epoch!")
        break

    avg_loss = total_loss / valid_batches
    print(f"\nEpoch {epoch+1} Loss: {avg_loss:.4f} (Valid batches: {valid_batches}/{len(train_loader)})")

    # Save checkpoint
    if (epoch + 1) % 2 == 0 and not torch.isnan(torch.tensor(avg_loss)):
        model.save_pretrained(f"git_xray_epoch{epoch+1}")
        processor.save_pretrained(f"git_xray_epoch{epoch+1}")
        print(f"✅ Saved checkpoint")

    # # Test generation
    # if (epoch + 1) % 2 == 0:
    #     print("\n=== SAMPLE GENERATION ===")
    #     sample = val_df.sample(1).iloc[0]
    #     try:
    #         pred = generate_report(model, processor, sample["image_path"], max_length=100)
    #         print(f"GT:   {sample['report_text'][:150]}")
    #         print(f"PRED: {pred[:150]}")
    #     except Exception as e:
    #         print(f"Generation failed: {e}")
    #     print("=" * 50 + "\n")

Trainable parameters: 176,619,066


Epoch 1: 100%|██████████| 747/747 [05:51<00:00,  2.12it/s, loss=2.2785, avg_loss=5.1241, lr=1.00e-05, grad_norm=13.67]



Epoch 1 Loss: 5.1241 (Valid batches: 747/747)


Epoch 2: 100%|██████████| 747/747 [05:08<00:00,  2.42it/s, loss=3.5711, avg_loss=2.8671, lr=8.89e-06, grad_norm=8.45]



Epoch 2 Loss: 2.8671 (Valid batches: 747/747)
✅ Saved checkpoint


...


Epoch 8 Loss: 1.8587 (Valid batches: 747/747)
✅ Saved checkpoint


Epoch 9: 100%|██████████| 747/747 [05:06<00:00,  2.44it/s, loss=2.2848, avg_loss=1.7893, lr=1.11e-06, grad_norm=9.63]



Epoch 9 Loss: 1.7893 (Valid batches: 747/747)


Epoch 10: 100%|██████████| 747/747 [05:06<00:00,  2.44it/s, loss=2.8997, avg_loss=1.7347, lr=0.00e+00, grad_norm=13.46]



Epoch 10 Loss: 1.7347 (Valid batches: 747/747)
✅ Saved checkpoint


In [ ]:
def generate_report(model, processor, image_path, max_length=150):
    """
    Generate a radiology report from a chest X-ray image.

    Args:
        model: Trained GIT model
        processor: GIT processor
        image_path: Path to the X-ray image
        max_length: Maximum length of generated report

    Returns:
        Generated report text
    """
    model.eval()

    # Load and process image
    image = Image.open(image_path).convert("RGB")

    # Process image with GIT processor
    inputs = processor(images=image, return_tensors="pt").to(model.device)

    # Generate report
    with torch.no_grad():
        generated_ids = model.generate(
            pixel_values=inputs.pixel_values,
            max_length=max_length,
            num_beams=4,  # Beam search for better quality
            no_repeat_ngram_size=3,  # Prevent repetition
            length_penalty=1.0,  # Encourage longer outputs
            early_stopping=True
        )

    # Decode generated tokens
    generated_text = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]

    return generated_text.strip()


# Load your trained model
def load_trained_model(checkpoint_path="git_xray_epoch10"):
    """
    Load the trained model from checkpoint.

    Args:
        checkpoint_path: Path to saved model directory

    Returns:
        model, processor
    """
    processor = AutoProcessor.from_pretrained(checkpoint_path)
    model = AutoModelForCausalLM.from_pretrained(checkpoint_path).cuda()
    model.eval()

    print(f"✅ Model loaded from {checkpoint_path}")
    return model, processor


# Example usage
if __name__ == "__main__":
    # Load your trained model
    model, processor = load_trained_model("git_xray_epoch10")  # Change to your checkpoint

    # Single image inference
    image_path = "/content/iu_xray/images/images_normalized/1000_IM-0003-2001.dcm.png"
    report = generate_report(model, processor, image_path)
    print(f"Generated Report:\n{report}")

✅ Model loaded from git_xray_epoch10
Generated Report:
size normal lungs clear noneothox plalff oreur eusion negative acuteiolmryity


In [ ]:
def test_on_validation_set(model, processor, val_df, num_samples=5):
    """
    Generate reports for validation samples and compare with ground truth.
    """
    model.eval()

    samples = val_df.sample(num_samples)

    for idx, row in samples.iterrows():
        print("=" * 80)
        image_path = row["image_path"]
        ground_truth = row["report_text"]

        # Generate prediction
        prediction = generate_report(model, processor, image_path, max_length=150)

        print(f"Image: {image_path}")
        print(f"\nGround Truth:\n{ground_truth}")
        print(f"\nPredicted Report:\n{prediction}")
        print("=" * 80 + "\n")

# Run test
test_on_validation_set(model, processor, val_df, num_samples=5)

Image: /content/iu_xray/images/images_normalized/2562_IM-1065-3001.dcm.png

Ground Truth:
the heart size is normal. the mediastinal contour is within normal limits. low lung volumes and bronchovascular crowding. mild bibasilar opacities, xxxx atelectasis. the lungs are free of any focal infiltrates. there are no nodules or masses. no visible pneumothorax. no visible pleural fluid. stable degenerative change throughout the thoracic spine. stable thoracolumbar retrolisthesis. there is no visible free intraperitoneal air under the diaphragm. no acute radiographic cardiopulmonary process.

Predicted Report:
heart andsti cons normal lungs clear

Image: /content/iu_xray/images/images_normalized/3360_IM-1613-1002.dcm.png

Ground Truth:
the lungs are clear bilaterally. specifically, no evidence of focal consolidation, pneumothorax, or pleural effusion.. left basilar subsegmental atelectasis versus scar noted. cardio mediastinal silhouette is unremarkable. visualized osseous structures of the t

In [ ]:
def generate_report_advanced(model, processor, image_path, strategy="beam_search"):
    """
    Generate report with different decoding strategies.

    Args:
        strategy: "beam_search", "greedy", "sampling", or "top_k"
    """
    model.eval()
    image = Image.open(image_path).convert("RGB")
    inputs = processor(images=image, return_tensors="pt").to(model.device)

    with torch.no_grad():
        if strategy == "beam_search":
            generated_ids = model.generate(
                pixel_values=inputs.pixel_values,
                max_length=150,
                num_beams=5,
                no_repeat_ngram_size=3,
                early_stopping=True
            )

        elif strategy == "greedy":
            generated_ids = model.generate(
                pixel_values=inputs.pixel_values,
                max_length=150,
                do_sample=False
            )

        elif strategy == "sampling":
            generated_ids = model.generate(
                pixel_values=inputs.pixel_values,
                max_length=150,
                do_sample=True,
                temperature=0.7,
                top_p=0.9,
                no_repeat_ngram_size=3
            )

        elif strategy == "top_k":
            generated_ids = model.generate(
                pixel_values=inputs.pixel_values,
                max_length=150,
                do_sample=True,
                top_k=50,
                no_repeat_ngram_size=3
            )

    generated_text = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]
    return generated_text.strip()

# Compare different strategies
image_path = val_df.sample(1).iloc[0]["image_path"]
print("Beam Search:", generate_report_advanced(model, processor, image_path, "beam_search"))
print("\nGreedy:", generate_report_advanced(model, processor, image_path, "greedy"))
print("\nSampling:", generate_report_advanced(model, processor, image_path, "sampling"))

Beam Search: cardedtina silhouette pulmonaryscuure within limits there noeur eusion pumra orneothox there no air opity plalff. is focal spaceac to aci granma therta torous there no cardpuona abnormal. are deerative of thoric. acuteiolmry.

Greedy: size normal cardedtina silhouette pulmonaryscuure normal lungs clear noneothox plalff. plalff is. focal opity theac, focal opity suggest previousuloous. are in right lobeac,xxlesis scar orlesis no cardpuona abnormal. acuteiolmryity

Sampling: size cardedtina silhouette pulmonaryscuure within limits normal. are of disease theac ao. istu of thoricrta noneothox plalff. are focal opities noted no plal. plal parchy is. are. acute.. acuteiolmry.


In [ ]:
!pip install rouge

In [ ]:
from nltk.translate.bleu_score import sentence_bleu, corpus_bleu
from rouge import Rouge

def evaluate_model(model, processor, val_df, num_samples=100):
    """
    Evaluate model using BLEU and ROUGE metrics.
    """
    model.eval()

    references = []
    hypotheses = []

    samples = val_df.sample(min(num_samples, len(val_df)))

    for idx, row in tqdm(samples.iterrows(), total=len(samples), desc="Evaluating"):
        # Generate prediction
        pred = generate_report(model, processor, row["image_path"], max_length=150)

        # Get ground truth
        gt = row["report_text"]

        references.append([gt.split()])  # BLEU expects list of references
        hypotheses.append(pred.split())

    # BLEU score
    bleu_score = corpus_bleu(references, hypotheses)

    # ROUGE score
    rouge = Rouge()
    rouge_scores = rouge.get_scores(
        [" ".join(h) for h in hypotheses],
        [" ".join(r[0]) for r in references],
        avg=True
    )

    print(f"\n{'='*50}")
    print(f"Evaluation Results on {len(samples)} samples:")
    print(f"{'='*50}")
    print(f"BLEU Score: {bleu_score:.4f}")
    print(f"ROUGE-1 F1: {rouge_scores['rouge-1']['f']:.4f}")
    print(f"ROUGE-2 F1: {rouge_scores['rouge-2']['f']:.4f}")
    print(f"ROUGE-L F1: {rouge_scores['rouge-l']['f']:.4f}")
    print(f"{'='*50}\n")

    return bleu_score, rouge_scores

# Run evaluation
# Install dependencies first: pip install nltk rouge
bleu, rouge = evaluate_model(model, processor, val_df, num_samples=100)


Evaluating: 100%|██████████| 100/100 [02:29<00:00,  1.49s/it]


Evaluation Results on 100 samples:
BLEU Score: 0.0000
ROUGE-1 F1: 0.1710
ROUGE-2 F1: 0.0066
ROUGE-L F1: 0.1645




/usr/local/lib/python3.12/dist-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.12/dist-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
